In [222]:
import requests
from bs4 import BeautifulSoup
import csv

# 알라딘 주간 베스트 100권 크롤링
# 2/2 ~ 2/8 1주 기준

# 책 정보 추출 함수
def extract_book(html):  
    subject=html.select_one("div.ss_book_list a.bo3").text
    
    info=html.select("div.ss_book_list li")[0].text
    
    if info=="ePub"or info=="PDF" or info=="무료PDF" or info=="대여ePub":
        
        info=html.select("div.ss_book_list li")[2].text
        
    else:
        
        info=html.select("div.ss_book_list li")[3].text
        
    writer=info.split('|')[0]
    
    publisher=info.split('|')[1]
    
    date=info.split('|')[2]
    
    stars=html.select_one("div.ss_book_list img")
    
    star=0
    
    if stars==None:
        
        star=0
        
    elif stars['src']=="//image.aladin.co.kr/img/common/star_s10.gif":
        
        star=5
        
    elif stars['src']=="//image.aladin.co.kr/img/common/star_s9.gif":
        
        star=4.5
        
    elif stars['src']=="//image.aladin.co.kr/img/common/star_s8.gif":
        
        star=4
        
    elif stars['src']=="//image.aladin.co.kr/img/common/star_s7.gif":
        
        star=3.5
        
    elif stars['src']==f"//image.aladin.co.kr/img/common/star_s6.gif":
        
        star=3
        
    elif stars['src']==f"//image.aladin.co.kr/img/common/star_s5.gif":
        
        star=2.5

    elif stars['src']==f"//image.aladin.co.kr/img/common/star_s4.gif":
        
        star=2
    
    return {
        'subject': subject,
        'writer':writer,
        'publisher':publisher,
        'date':date,
        'star':star
    }

# 각 페이지마다 책 정보 추출 함수
def extract_books(page_count):
    books = []
    
    for page in range(page_count):
    
        print(f"Scrapping page {page+1}")
        
        rs = requests.get(f"https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=EBookBestseller&BranchType=9&CID=0&page={page+1}&cnt=300&SortOrder=1")
        
        soup = BeautifulSoup(rs.text, "html.parser")
        
        results=soup.find_all("div",class_="ss_book_box")
        
        for rs in results:
            
            book = extract_book(rs)
            
            books.append(book)
            
    return books

# CSV 저장
# 1-50 / 51-100 두 페이지만 진행
books=extract_books(2)

file=open("aladin_best100.csv",mode="w")

write=csv.writer(file)

write.writerow(["subject", "writer", "publisher","date", "star"])

for book in books:
    
    write.writerow(list(book.values()))

Scrapping page 1
Scrapping page 2


In [223]:
import pandas as pd

df=pd.read_csv("aladin_best100.csv",encoding='cp949')

In [224]:
df

,subject,writer,publisher,date,star
0,아버지의 해방일지,정지아 (지은이),창비,2022년 9월,5.0
1,음식 중독,"마이클 모스 (지은이), 연아람 (옮긴이)",민음사,2023년 1월,4.5
2,이토록 평범한 미래,김연수 (지은이),문학동네,2022년 10월,4.5
3,사랑의 이해,이혁진 (지은이),민음사,2019년 4월,4.0
4,무조건 합격하는 암기의 기술,이윤규 (지은이),더퀘스트,2023년 1월,5.0
...,...,...,...,...,...
95,타임퀘이크,"커트 보니것 (지은이), 유정완 (옮긴이)",문학동네,2023년 1월,5.0
96,인간실격 (한글판),"다자이 오사무 (지은이), 김소영 (옮긴이)",미르북컴퍼니,2013년 12월,4.0
97,거의 모든 행동 표현의 영어,서영조 (지은이),사람in,2022년 3월,5.0
98,긴긴밤,루리 (지은이),문학동네,2021년 2월,5.0
